In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [3]:
!wget https://raw.githubusercontent.com/futurexskill/bigdata/refs/heads/master/bank_prospects.csv

--2025-01-06 10:44:05--  https://raw.githubusercontent.com/futurexskill/bigdata/refs/heads/master/bank_prospects.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306 [text/plain]
Saving to: ‘bank_prospects.csv’

bank_prospects.csv  100%[===================>]     306  --.-KB/s    in 0s      

2025-01-06 10:44:06 (4.61 MB/s) - ‘bank_prospects.csv’ saved [306/306]



In [4]:
df = spark.read.csv("bank_prospects.csv", header=True, inferSchema=True)


In [5]:
df.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  NULL|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  NULL|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|NULL| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
|  27| 37000|Female|unknown|        N|
+----+------+------+-------+---------+



In [6]:
filtered_df = df.filter(df['country'] != "unknown")

In [7]:
filtered_df.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  NULL|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  NULL|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|NULL| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



In [8]:
filtered_df.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Purchased: string (nullable = true)



In [22]:
from pyspark.sql.functions import mean

In [23]:
mean_age1 = filtered_df.select(mean(filtered_df["age"])).collect()

In [24]:
type(mean_age1)

list

In [26]:
mean_age = mean_age1[0][0]
mean_age

22.11111111111111

In [27]:
from pyspark.sql.types import IntegerType,FloatType

In [28]:
df2 = filtered_df.withColumn("salary", filtered_df["salary"].cast(FloatType()))

In [29]:
mean_salary = df2.select(mean(df2["salary"])).collect()

In [30]:
mean_sal=mean_salary[0][0]

In [31]:
mean_sal

31875.0

In [32]:
final_df = df2.na.fill(mean_age,["age"])

In [33]:
final_df = final_df.na.fill(mean_sal,["salary"])

In [34]:
final_df.show()

+---+-------+------+-------+---------+
|Age| salary|Gender|Country|Purchased|
+---+-------+------+-------+---------+
| 18|20000.0|  Male|Germany|        N|
| 19|22000.0|Female| France|        N|
| 20|24000.0|Female|England|        N|
| 21|31875.0|  Male|England|        N|
| 22|50000.0|  Male| France|        Y|
| 23|35000.0|Female|England|        N|
| 24|31875.0|  Male|Germany|        N|
| 25|32000.0|Female| France|        Y|
| 22|35000.0|  Male|Germany|        N|
| 27|37000.0|Female| France|        N|
+---+-------+------+-------+---------+

